in last router , we invoked model and call a tool, and it return a tool message.
but now we would like to pass tool message directly to model again.
then model will decide if it need to call another tool or response directly to request. 

ReAct

act > observe > reason

act > call a specific tool
observe > pass the tool output back to the model
reason > let the model reason about the tool output to decide whet to do next

In [9]:
## Part 1 - set up LLM
import os
from dotenv import load_dotenv
load_dotenv()

#os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")


from langchain_groq import ChatGroq
llm = ChatGroq(model="deepseek-r1-distill-llama-70b")
llm

# from langchain_openai import ChatOpenAI
# llm_openai = ChatOpenAI(model="gpt-3.5-turbo")
# llm_openai

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x10fe321b0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x10fe32f30>, model_name='deepseek-r1-distill-llama-70b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [10]:
def multiply(a:int, b:int) -> int:
    """ Multiply a and b.
    args:
    a: first int
    b: second int
    """
    return a * b

def add(a: int, b: int) -> int:
    """
    Adds two integers and returns the result.

    Args:
        a (int): The first integer.
        b (int): The second integer.

    Returns:
        int: The sum of a and b.
    """
    return a + b

def divide(a: int, b: int) -> float:
    """
    Divides two integers and returns the result.

    Args:
        a (int): The numerator.
        b (int): The denominator.

    Returns:
        float: The result of division.

    Raises:
        ValueError: If the denominator is zero.
    """
    return a / b


In [3]:
tools=[add,multiply,divide]
llm_with_tools =llm.bind_tools(tools, parallel_tool_calls=False)

In [12]:
from typing_extensions import TypedDict
from langchain_core.messages import AnyMessage
from typing import Annotated
from langgraph.graph.message import add_messages 

class MessageState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

In [13]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

sys_msg = SystemMessage(
    content="You are a helpful assistant that can perform basic arithmetic operations. "
            "You can add, multiply, and divide numbers. "
            "You can also use the tools to perform these operations."
)

def assistant(state: MessageState):
      return{"messages":[llm_with_tools.invoke([sys_msg]+state["messages"])]}

In [15]:
##Add tool as a node
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition
from IPython.display import Image, display
from langgraph.graph import StateGraph, START, END

builder = StateGraph(MessageState)

builder.add_node("assistant", assistant)
builder.add_node("tools",ToolNode(tools))

builder.add_edge(START, "assistant")
builder.add_conditional_edges(
      "assistant",
      # if the latest message (result) from assistant is a tool call -> tools condition routes to tools
      #if the latest message (result) from assistant is not a tool call -> tools condition routes to END

      tools_condition,
)

builder.add_edge("tools", "assistant")

graph=builder.compile()


In [7]:
from IPython.display import Image,display
display(Image(graph.get_graph().draw_mermaid_png()))

ReadTimeout: HTTPSConnectionPool(host='mermaid.ink', port=443): Read timed out. (read timeout=10)

In [ ]:
messages = [HumanMessage(content="What is 2+2*5/1"),]
messages= graph.invoke({"messages": messages})
for m in messages['messages']:
    m.pretty_print()

In [ ]:
messages = [HumanMessage(content="multiply result by 2"),]
messages= graph.invoke({"messages": messages})
for m in messages['messages']:
    m.pretty_print()

Memory in agents gents using Memory Saver

the above step multiply result with 2 is not working , as their is no history management done. We are saving previous message . So for testing and debugging purpouse we are using in -memory saving library i.e. MemorySaver(), Now using using this we need to create a config, where we will manage the session id

In [22]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()
react_graph = builder.compile(checkpointer=memory)

In [24]:
config = {"configurable":{"thread_id":"1"}}
messages = [HumanMessage(content="What is 2+2*5/1"),]

messages= react_graph.invoke({"messages": messages},config)
#config is maintaining session id 
for m in messages['messages']:
    m.pretty_print()



          

================================ Human Message =================================

What is 2+2*5/1
================================== Ai Message ==================================
Tool Calls:
  add (call_knqv)
 Call ID: call_knqv
  Args:
    a: 2
    b: 10
================================= Tool Message =================================
Name: add

12
================================== Ai Message ==================================
Tool Calls:
  multiply (call_972f)
 Call ID: call_972f
  Args:
    a: 2
    b: 5
================================= Tool Message =================================
Name: multiply

10
================================== Ai Message ==================================

<tool_call>{"id":"call_5e3f","name":"divide","arguments":{"a":10,"b":1}}<｜tool▁calls▁end｜> 2+2*5/1 = 12


In [25]:
messages = [HumanMessage(content="multiply result by 2"),]
messages= react_graph.invoke({"messages": messages},config)
#config is maintaining session id
for m in messages['messages']:
    m.pretty_print()

================================ Human Message =================================

What is 2+2*5/1
================================== Ai Message ==================================
Tool Calls:
  add (call_knqv)
 Call ID: call_knqv
  Args:
    a: 2
    b: 10
================================= Tool Message =================================
Name: add

12
================================== Ai Message ==================================
Tool Calls:
  multiply (call_972f)
 Call ID: call_972f
  Args:
    a: 2
    b: 5
================================= Tool Message =================================
Name: multiply

10
================================== Ai Message ==================================

<tool_call>{"id":"call_5e3f","name":"divide","arguments":{"a":10,"b":1}}<｜tool▁calls▁end｜> 2+2*5/1 = 12
================================ Human Message =================================

multiply result by 2
================================== Ai Message ==================================

To calculate \( 